#### Domain YG에서 만든 코드를 전체 파일에 적용 시킨다. 각각의 DataFrame에 저장시킨뒤 비교
#### 딕셔너리를 이용하여 가중치를 합치는 작업을 거친다. Score = sum(tf_rof)/sum(tf) / 이기 때문에 Key값을 word로 주고 value 값을 sum(tf_rof) , sum(tf) 값으로 주면 좋겠다.
#### 명사만 추출 했으나 추후에 서술어까지 해야함 .
#### 매일매일 등락률 계산을 위해서는 등락률 계산 모듈이 필요함 Sentiment_Dic_Domain file 참고 


In [5]:
import datetime
from datetime import date
import pandas as pd
import re 

from konlpy.tag import Komoran
from collections import Counter

In [6]:
def getWeekday(start, end):
#start = '20200411' , end = '20200922'
    # 평일을 구하기 위한 datetime의 기본적인 변수를 생성해준다. 
    s_year = int(start[0:4]) # 2020
    s_month = int(start[5:6]) # 4
    s_day = int(start[6:8]) # 11

    e_year = int(end[0:4]) # 2020
    e_month = int(end[5:6]) # 9
    e_day = int(end[6:8]) # 22 

    day_delta = datetime.timedelta(days=1) 
    start_date = datetime.date(s_year, s_month, s_day)
    end_date = datetime.date(e_year, e_month, e_day)
    length = (end_date - start_date).days # 시작일 - 끝나는일의 일 수 
 
    # 평일 데이터만 가져오기 위해 위에서 짠 것들을 합쳐준다.
    date_weekday = [] # 평일날짜를 넣기위한 리스트 
    for i in range(length+1) : # 마지막날을 포함하기 위해 length에 + 1 을 해준다. 
        date = start_date + i*day_delta # date 는 for 문을 돌며 day_delta로 1일 씩 더해지게 된다. 
        date_w = date.weekday() # date에 저장된 날짜를 weekday로 본다 0~6의 값을 가진다. 
        if date_w < 5 : # 조건문을 걸어 0~4 월~금 만 date_weekday 리스트에 추가하도록한다. 
            date_weekday.append(str(date))
            
    return date_weekday # 평일만 포함된 date weekday 리스트가 나온다. 이를 이용해 평일 엑셀 데이터만 불러 오도록 한다. 


# 년/월/일 변수가 / 로 구분 되어있다면 data name 과 맞춰주기 위해 /를 -로 바꿔주는 부분. 
def replaceSlash(df) :
    dates = []
    df_date = df.iloc[:,:1]
    df_date = df_date.sort_values(by='date',ascending=False)
    list1 = df_date.values.tolist()
    for i in range(len(df_date)):
        tokens = list1[i][0]
        token = tokens.split('-')
        year = token[0]
        month = token[1]
        #month = month.replace('0','')
        day = token[2]
        date = year + '-' + month + '-' + day 
        dates.append(date)
    return dates

# 문장내 모든 conma를 제거해주는 함수 
def replaceConma(df, start, finish) : # start, finish 열의 인덱스를 지정해 주면된다. 
    for j in range(start, finish+1):
        for i in range(len(df.iloc[:,j])):
            df.iloc[:,j][i] = df.iloc[:,j][i].replace(",", "")
    return df
# df = replaceAllConma(df, 1, 9) 1열 부터 9열 까지의 모든 데이터의 conma를 삭제.


# user dic에 한글자가 있는 경우 이를 삭제해주는 함수 . filename 에는 userdic name 이 들어간다. 
def delDicOneword(filename) :
    with open(f'{filename}.txt', 'r' ) as f :
        line = f.readlines()
        print(line)

    # 개행문자를 포함하여 글자수가 2개 이하인 모든 글자를 삭제. 
    for i in line :
        i.strip()
        if len(i) < 3 :
            line.remove(i)

    with open(f'{filename}.txt', 'w') as f:
        for i in line:
            f.write(i)

# tf score 계산

def getTfrofScore(date) :
# date = '2020-04-13' 

    df1 = pd.read_excel(f'./data_news/YG_DATAyg {date}.xlsx')
    sentences = [] 
    list1 = list(set(df1['title'].values.tolist())) # 중복을 제거 한다.
    pattern = '[^\w\s]'
    repl = ''
    for i in list1 :
        sen = i.split(']')
        if len(sen) >= 2 : # 스플릿 했을때 ] 가 있으면 len이 2 가 나오고 없으면 1이 나옴. 그중 문장을 선택하게 조건문을 부여
            sen = sen[1]
        else :
            sen = sen[0]
        sen = re.sub(pattern," ",sen) # pattern에서 정의한 특수문자 정규표현식 을 제거 
        sen = sen.strip() # 양옆의 공백을 제거 

        sentences.append(sen)
    sentences

    komoran_userdic=Komoran(userdic='./userdicCanSur_YG_Sentiment.txt')
    corpus = sentences
    wordCount = {}
    wordlist = []
    answer = []
    for i in corpus:
        wordlist.append(komoran_userdic.nouns(i))
    for i in wordlist:
        answer += i
    #print(answer)


    # 빈도수가 2번 이상인 단어만을 대상으로 한다. 

    attention = {}
    attention = Counter(answer)
    df1 = pd.DataFrame(attention.keys())
    df1.columns =['Name']
    df1['Frequency'] = pd.DataFrame(attention.values())
    df1 = df1[df1['Frequency'] >= 2] 
    df1.reset_index(inplace = True)
    df1 = df1.drop('index', axis =1)

    # 한글자인 데이터를 삭제 해준다.
    for i in range(len(df1)):
        word = df1['Name'][i] # iloc로 하면 df를 drop 했을때 키를 못찾는 에러가 발생함. 
        print(i, word)
        if len(word) < 2 : # Name 테이블 안에 글자가 한개이면 이를 삭제
            print(i, word)
            df1.drop(index=i, inplace=True)


    df = df[ df['date'] == date ] # 조건문을 걸어 변수 date 와 같은날의 행의 모든 데이터를 반환 
    rof = df['rof'].iloc[0] # 그날의 데이터중 rof값을 반환
    
    df1['tf_rof'] = df1['Frequency']*rof # 빈도수 dataFrame에 df_rof 값을 저장
    return df1

1. stock data 를 불러온다 .
2. 모든 데이터의 conma를 삭제한다 .
3. stock data 의 년월일을 news data 년월일과 맞추기 위해 slash를 삭제하고 hyphen을 넣는다.  
##### 미구현 / 등락률을 계산한다. 계산후 na 값은 drop 한다.(가장 첫날에 해당함)
4. news data 를 불러온다.
##### 서술어 미구현 / 사전 정의된 Komoran 형태소 분석기를 이용해 뉴스 명사(서술어)의 Tf를 계산한다.
5. tfrof score를 구하기 위해 stock data에서 new data의 date에 맞는 등락률을 가져와 Tf 에 곱한다. 


#### date weekday 에서 day 리스트를 뽑아서 2020-04-11의 형태로 뽑아 for 문을 돌린다. date 만 넣으면 자동으로 tfrof score를 계산할 수 있도록 구현 

In [9]:
def main():
    # csv 파일로 들어가 등락률을 미리 계산해주어야함.  등락률 = 당일대비 / 전일종가
    df = pd.read_csv('./data_stock/YG_1y.csv', encoding = 'ISO-8859-1')
    df.columns = ['date', 'closing', 'diff', 'volume', 'transaction', 'market', 'high', 'low', 'capital', 'stocks', 'rof']
    df = replaceConma(df, 1, 9)
    #df = replaceSlash(df) # stock data date가 2020/1/3 일때 사용
    
    df1 = getTfrofScore('2020-04-11')
    days = getWeekday('20200411', '20200922') # 내가 기준으로 한 범위
    for day in days
        getTfrofScore(day) # 이렇게 for문을 돌리면 각 날짜에 대한 TfrofScore df 를 반환함 
    

SyntaxError: invalid syntax (<ipython-input-9-8007a541f240>, line 9)

In [8]:
if __name__ == "__main__" :
    main()

FileNotFoundError: [Errno 2] No such file or directory: './data_news/YG_DATAyg 2020-04-11.xlsx'

In [10]:
getWeekday('20200411', '20200922')

['2020-04-13',
 '2020-04-14',
 '2020-04-15',
 '2020-04-16',
 '2020-04-17',
 '2020-04-20',
 '2020-04-21',
 '2020-04-22',
 '2020-04-23',
 '2020-04-24',
 '2020-04-27',
 '2020-04-28',
 '2020-04-29',
 '2020-04-30',
 '2020-05-01',
 '2020-05-04',
 '2020-05-05',
 '2020-05-06',
 '2020-05-07',
 '2020-05-08',
 '2020-05-11',
 '2020-05-12',
 '2020-05-13',
 '2020-05-14',
 '2020-05-15',
 '2020-05-18',
 '2020-05-19',
 '2020-05-20',
 '2020-05-21',
 '2020-05-22',
 '2020-05-25',
 '2020-05-26',
 '2020-05-27',
 '2020-05-28',
 '2020-05-29',
 '2020-06-01',
 '2020-06-02',
 '2020-06-03',
 '2020-06-04',
 '2020-06-05',
 '2020-06-08',
 '2020-06-09',
 '2020-06-10',
 '2020-06-11',
 '2020-06-12',
 '2020-06-15',
 '2020-06-16',
 '2020-06-17',
 '2020-06-18',
 '2020-06-19',
 '2020-06-22',
 '2020-06-23',
 '2020-06-24',
 '2020-06-25',
 '2020-06-26',
 '2020-06-29',
 '2020-06-30',
 '2020-07-01',
 '2020-07-02',
 '2020-07-03',
 '2020-07-06',
 '2020-07-07',
 '2020-07-08',
 '2020-07-09',
 '2020-07-10',
 '2020-07-13',
 '2020-07-